In [1]:
import requests
import json
import time
import pandas as pd
from datetime import datetime, timedelta

In [3]:
def target_date_to_datetime(target_date, date_format="%Y-%m-%d"):
    target_datetime = datetime.strptime(target_date, date_format)
    return target_datetime


def datetime_to_timestamp(date_as_datetime):
    date_as_timestamp = int(datetime.timestamp(date_as_datetime))
    return date_as_timestamp


def get_yesterday_datetime(date_as_datetime, days=1):
    yesterday_datetime = date_as_datetime - timedelta(days)
    return yesterday_datetime


def get_steam_api_url(app_id):
    return f"https://store.steampowered.com/appreviews/{app_id}"


def get_request_params(target_timestamp):
    # References:
    # - https://partner.steamgames.com/doc/store/getreviews
    # - browser dev tools on store pages, e.g. https://store.steampowered.com/app/570/#app_reviews_hash

    if target_timestamp is None:
        raise Exception("Empty target_timestamp!")

    params = {
        "json": "1",
        "num_per_page": "0",  # text content of reviews is not needed
        # caveat: default seems to be "english", so reviews would be missing if unchanged!
        "language": "all",
        # caveat: default is "steam", so reviews would be missing if unchanged!
        "purchase_type": "all",
        # to un-filter review-bombs, e.g. https://store.steampowered.com/app/481510/
        "filter_offtopic_activity": "0",
        "start_date": "1",  # this is the minimal value which allows to filter by date
        "end_date": str(target_timestamp),
        "date_range_type": "include",  # this parameter is needed to filter by date
    }

    return params


def download_review_stats(app_id, target_timestamp):
    url = get_steam_api_url(app_id)
    params = get_request_params(target_timestamp)

    response = requests.get(url, params=params)

    if response.ok:
        result = response.json()
    else:
        result = None

    return result


In [14]:
app_id = 1182900

In [36]:
rate_limits = {
    "max_num_queries": 600,
    "cooldown": (4 * 60) + 10,  # 4 minutes plus a cushion,
}

target_date = "2022-10-16"
date_format = "%Y-%m-%d"

target_datetime = target_date_to_datetime(target_date=target_date, date_format=date_format)
target_timestamp = datetime_to_timestamp(target_datetime)

res = download_review_stats(app_id=app_id, target_timestamp=target_timestamp)

In [37]:
fields = ['total_reviews', 'total_positive', 'total_negative', 'review_score']
query_summary = res["query_summary"]
review = dict()
for key in fields:
    review[key] = query_summary[key]

review

{'total_reviews': 0,
 'total_positive': 0,
 'total_negative': 0,
 'review_score': 0}

In [33]:
target_datetime.strftime(date_format)

'2022-10-21'

# Test

In [4]:
app_id = 1182900
target_date = "2022-10-20"
date_format = "%Y-%m-%d"


def get_info(res):
    fields = ['total_reviews', 'total_positive',
              'total_negative', 'review_score']
    query_summary = res["query_summary"]
    info = dict()
    for key in fields:
        info[key] = query_summary[key]
    return info


def test(app_id, target_date, date_format):
    target_datetime = target_date_to_datetime(target_date, date_format)
    target_timestamp = datetime_to_timestamp(target_datetime)
    data = dict()
    res = download_review_stats(app_id, target_timestamp)
    if not res:
        print(f"Cannot get {app_id} !!!")
        return
    info = get_info(res)

    while (info.get('total_reviews')):
        data[target_datetime.strftime(date_format)] = info
        target_datetime = get_yesterday_datetime(target_datetime)
        target_timestamp = datetime_to_timestamp(target_datetime)
        res = download_review_stats(app_id, target_timestamp)
        info = get_info(res)

    return data


In [5]:
res = test(app_id, target_date, date_format)

In [6]:
res

{'2022-10-20': {'total_reviews': 763,
  'total_positive': 659,
  'total_negative': 104,
  'review_score': 8},
 '2022-10-19': {'total_reviews': 452,
  'total_positive': 392,
  'total_negative': 60,
  'review_score': 8},
 '2022-10-18': {'total_reviews': 75,
  'total_positive': 59,
  'total_negative': 16,
  'review_score': 6}}

In [48]:
df = pd.DataFrame.from_dict(res, orient='index')
df.index.name = "Date"

In [49]:
df.to_csv("test.csv")

In [50]:
df

,total_reviews,total_positive,total_negative,review_score
Date,,,,
2022-10-20,761,652,109,8
2022-10-19,452,392,60,8
2022-10-18,75,58,17,6


In [1]:
import steamreviews

In [6]:
review_dict, query_count = steamreviews.download_reviews_for_app_id(374320)

[appID = 374320] expected #reviews = 336685
Number of queries 150 reached. Cooldown: 310 seconds
Number of queries 150 reached. Cooldown: 310 seconds
Number of queries 150 reached. Cooldown: 310 seconds
Number of queries 150 reached. Cooldown: 310 seconds
Number of queries 150 reached. Cooldown: 310 seconds


In [5]:
review_dict

{'reviews': {}, 'query_summary': {'total_reviews': -1}, 'cursors': {}}